# Example: Fast and accurate prediction of the regioselectivity of electrophilic aromatic substitution reactions

RegioSQM method protonates all aromatic C–H carbon atoms and identifies those with the lowest free energies in **solvent** using the semiempirical quantum chemical **method** as the most nucleophilic center.

As per the Regio2020 version, in this example we are using
**xTB GFN1** in **Methanol**

Reference
- https://doi.org/10.1039/C7SC04156J
- https://doi.org/10.1186/s13321-021-00490-7
- https://github.com/jensengroup/regiosqm
- https://github.com/NicolaiRee/RegioSQM20


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging
import sys

In [ ]:
from tqdm import tqdm

In [ ]:
tqdm.pandas()  # Show progress bars on pandas functions

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import SVG
from rdkit import Chem
from rdkit.Chem import AllChem, PandasTools
from rdkit.Chem.Draw import MolsToGridImage, MolToImage, rdMolDraw2D

In [ ]:
try:
    import ppqm
except ModuleNotFoundError:
    import pathlib

    cwd = pathlib.Path().resolve().parent
    sys.path.append(str(cwd))
    import ppqm

In [ ]:
# Set logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger("ppqm").setLevel(logging.INFO)
logging.getLogger("xtb").setLevel(logging.INFO)
show_progress = True

In [ ]:
# Set DataFrames visuals
PandasTools.RenderImagesInAllDataFrames(images=True)
pd.set_option('display.float_format','{:.2f}'.format)

## Define protonation reactions with SMARTS

In [ ]:
reaction1 = AllChem.ReactionFromSmarts("[C;R;H1:1]=[C,N;R;H1:2]>>[CH2:1][*H+:2]")
reaction2 = AllChem.ReactionFromSmarts("[C;R;H1:1]=[C,N;R;H0:2]>>[CH2:1][*+;H0:2]")

In [ ]:
reaction1

In [ ]:
reaction2

## Define a molecule you like

In [ ]:
smiles = "Cc1cc(NCCO)nc(-c2ccc(Br)cc2)n1"  # CHEMBL1956589
molobj = Chem.MolFromSmiles(smiles)

In [ ]:
molobj

In [ ]:
Chem.Kekulize(molobj, clearAromaticFlags=True)

## Protonate all aromatic carbons




In [ ]:
def get_target_atoms(molobj, target):
    """ Find target atom indices from SMART """
    atoms = molobj.GetSubstructMatches(target)
    # convert tuple of tuple to one-dimensional list
    atoms = [element for tupl in atoms for element in tupl]
    return atoms

In [ ]:
molobjs = []
target_atoms = []

smarts_1 = Chem.MolFromSmarts("[C;R;H1:1]=[C,N;R;H1:2]")
smarts_2 = Chem.MolFromSmarts("[C;R;H1:1]=[C,N;R;H0:2]")
atoms_1 = get_target_atoms(molobj, smarts_1)
atoms_2 = get_target_atoms(molobj, smarts_2)

i = 0
products_1 = reaction1.RunReactants((molobj,))
for x in products_1:

    molobj_prime = x[0]
    smiles = Chem.MolToSmiles(molobj_prime)
    smiles = smiles.replace("NH2+", "N+")
    molobj_prime = Chem.MolFromSmiles(smiles)

    molobjs.append(molobj_prime)
    target_atoms.append(atoms_1[i])

    i += 1

isav = i

products_2 = reaction2.RunReactants((molobj,))
for x in products_2:

    molobj_prime = x[0]
    smiles = Chem.MolToSmiles(molobj_prime)
    smiles = smiles.replace("NH2+", "N+")
    molobj_prime = Chem.MolFromSmiles(smiles)

    molobjs.append(molobj_prime)
    target_atoms.append(atoms_2[2 * (i - isav) - 2])

    i += 1

In [ ]:
MolsToGridImage(
    molobjs,
    molsPerRow=3,
    subImgSize=(250, 250),
    useSVG=True,
)

In [ ]:
[Chem.MolToSmiles(m) for m in molobjs]

## Now let's find out which are most stable using quantum chemistry

In [ ]:
df = pd.DataFrame(molobjs, columns=["molobj"])
df["atom_index"] = target_atoms

In [ ]:
df

In [ ]:
xtb = ppqm.xtb.XtbCalculator(scr="_tmp_directory_", n_cores=2, cmd="xtb", show_progress=False)
xtb

Let's define a function that we can map onto a pandas DataFrame on each row. We want to calculate the energy for each site which requires some conformer expansion. We are only interested in the lowest energy per conformer.

In [ ]:
def calculate_energy(molobj):
    """

    For each protonated molecule

    - Generate conformers (max 20 conformers)
    - Minimize all conformers
    - Get the energy for each conformer
    - Return the lowest energy

    """

    xtb_options = {
        "gfn": 1,
        "alpb": "Methanol",
        "opt": None,
    }

    # Generate conformers
    molobj = ppqm.tasks.generate_conformers(molobj, max_conformers=20)

    # Optimize with xTB
    results = xtb.calculate(molobj, xtb_options)

    # Collect energies and find lowest
    conformer_energies = [result["scc_energy"] for result in results]
    min_energy = np.min(conformer_energies)
    min_energy *= ppqm.units.hartree_to_kcalmol

    return min_energy

In [ ]:
# example usage: reference_energy = calculate_energy(molobj)

In [ ]:
df["energy"] = df["molobj"].progress_apply(calculate_energy)

In [ ]:
df["rel_energy"] = df["energy"].values - np.min(df["energy"].values)

In [ ]:
df

In [ ]:
# Define energy cutoffs
cutoff1 = 1.0  # kcal/mol
cutoff2 = 3.0  # kcal/mol

# Define pretty colors
colors = dict()
colors["green"] = (119, 198, 110)
colors["green"] = tuple(x/255 for x in colors["green"])
colors["red"] = (201, 43, 38)
colors["red"] = tuple(x/255 for x in colors["red"])

# Find reactive centers and convert index type to int.
# rdkit doesn't understand np.int
green_indices = df[df["rel_energy"] < cutoff1]["atom_index"].values
green_indices = [int(x) for x in green_indices]
red_indices = df[df["rel_energy"] < cutoff2]["atom_index"].values
red_indices = [int(x) for x in red_indices if x not in green_indices]

# All highlights
highlights = green_indices + red_indices

# Map highlight to a color
colormap = dict()
colormap.update({key: [colors["green"]] for key in green_indices})
colormap.update({key: [colors["red"]] for key in red_indices})

In [ ]:
# should be working, but does not respect colors
# MolToImage(
#    molobj,
#    highlightAtoms=highlights,
#    highlightMap=colormap,
#    size=(500,500),
# )

In [ ]:
# http://rdkit.blogspot.com/2020/04/new-drawing-options-in-202003-release.html
d2d = rdMolDraw2D.MolDraw2DSVG(500, 500)
d2d.DrawMoleculeWithHighlights(molobj, "Regioselective site(s)", dict(colormap), {}, {}, {})
d2d.FinishDrawing()
SVG(d2d.GetDrawingText())